## Land suitability for avocado in Morocco using national level data

Authors: Gianluca Franceschini (FAO, NSL), Shrijwal Adhikari (FAO, NSL)

Contact: gianluca.franceschini@fao.org / shrijwal.adhikari@fao.org

---

The present study assess the land suitable for avocado in Morocco using the similar approach assessed by Gianluca Franceschini for Fast track land suitability for avocado in Morocco. This assessment uses original vector data from the annalysis conducted by Rachid Moussadek and Hamza Iaaich in 2020 comprising the region of Rabat, Salé, Kénitra and the province of Larache. 

We used a similar approach for this region but converting vector to raster of resolution 60m. 60m was chosen as a balance between processing speed and size of the data itself. It uses land cover data, data relating with physiography (aspect and exposure) and soil characteristics (texture, drainage). These factors are matched against specific thresholds identifying optimal suitable, intermediate and not suitable conditions. Then, the individual layers were overlaid to get a final score for the whole area of interest. The purpose of the analysis was to use similar approach used for Fast track land suitability but using local data and tally the results. 

In [1]:
"""Import libraries"""
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from osgeo import gdal

In [2]:
"""Set parameters"""
base_path = r"/media/sri/Shrijwal/fao_shri_gis/loukkos_vector/raster60/AOI/raster" #Root to be updated if different
os.chdir(base_path)
ref_raster = './Suitability/reference.tif'

## Data sources

Data have been obtained from the project data of Rachid Moussadek and Hamza Iaaich in 2020.
The spatial resolution was set to 60m.

The data used for the analysis were:
1. Land cover
2. Exposition
3. Slope
4. Drainage
5. Texture

|**Data** |**Source/Provider** |**Resolution**| 
|:-- |:-- |:-- |
|Exposition |ASTER degital elevation model|30m|
|Slope |ASTER degital elevation model|30m|
|Soil |INRA (2000)|vector  1:500,000 scale|

In [ ]:
# """Read data"""

drainage = gdal.Open('AOI-drainage_26191.tif').ReadAsArray()
drainage = drainage[:,:-1] #Remove one line to have the same shape
exposure = gdal.Open('AOI-exposure_26191.tif').ReadAsArray()
exposure=exposure[:,:-1]
landc = gdal.Open('AOI-LC_26191.tif').ReadAsArray()
slope  = gdal.Open('AOI-slope_26191.tif').ReadAsArray()
texture  = gdal.Open('AOI-texture_26191.tif').ReadAsArray()
texture=texture[:,:-1]

In [4]:
"""Function to export the results"""
def saveRaster(ref_raster_path, out_path, numpy_raster):

    # Read random image to get projection data
    img = gdal.Open(ref_raster_path)
    # allocating space in hard drive
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(out_path, img.RasterXSize, img.RasterYSize, 1, gdal.GDT_Float32)
    # set image paramenters (infrormation related to cordinates)
    outdata.SetGeoTransform(img.GetGeoTransform())
    outdata.SetProjection(img.GetProjection())
    # write numpy matrix as new band and set no data value for the band
    outdata.GetRasterBand(1).WriteArray(numpy_raster)
    outdata.GetRasterBand(1).SetNoDataValue(0)
    # flush data from memory to hard drive
    outdata.FlushCache()
    outdata=None

## Crop parameters

A land suitabilty analysis matches specific crop tolerance for soil, terrain and climate with local prevailing conditions. In order to assess the suitable thresholds for avocado, the **ECOCROP** database was used to define these values. The thresholds are typically defined for optimum conditions where the crop growth is not constraint and can express its full potential, to moderate suitability where crop growth is somehow limited, to unsuitable conditions whereas specific values of a certain factor prevent the crop growth (irrespective from the values of the other parameters, that could be potentially suitable).

The following thresholds have been used for avocado:

|**Parameter** |**Optimum min** |**Optimum max** |**Absolute min**|**Absolute max**|
|:-- |:-- |:-- |:-- |:-- |
|Minimum Temperature |14°C |na|10°C |na|
|Maximum Temperature |na |40°C |na|45°C |
|Precipitation |500mm |2000mm |2500mm|300mm |
|Slope|less than 2° |na |na|15°|
|Aspect |All except South and South-East |na |na|na |
|Soil ph |5 |5.8 |4.5|7 |

## Land Cover

The assessment used local land cover data. It used seven different land cover classes.
Identifying values and their corresponding land cover classes are:

1 for Grandes Cultures et Maraichage/Field Crops and market gardening 
2 for Arboriculture /Arboriculture  
3 for Forêt/Forest 
4 for Etendue d'eau/Body of Water 
5 for Terrain Non Agricole/Non- agricultural land 
6 for Sol Nu/Bare ground, and
7 for Agriculture sous Serres/Greenhouse agriculture

In [ ]:
#Define the thresholds for land cover 
#1: Grandes Cultures et Maraichage/Field Crops and market gardening 2: Arboriculture /Arboriculture  
#3: Forêt/Forest 4: Etendue d'eau/Body of Water 5: Terrain Non Agricole/Non- agricultura land 
#6: Sol Nu/Bare ground  7: Agriculture sous Serres/Greenhouse agriculture

landc_suitability = landc.copy()
landc_suitability[(landc_suitability == 4) | (landc_suitability == 5)] = 0 #Not Suitable
landc_suitability[(landc_suitability == 2) | (landc_suitability == 3) | (landc_suitability == 6)] = 10 #Not Suitable
landc_suitability[(landc_suitability == 1) | (landc_suitability == 7)] = 20  #Highly Suitable


#Export results
saveRaster(ref_raster, "./Suitability/landc_suitability.tif", landc_suitability)

In [ ]:
"""Display results"""
cmap_suitability = colors.ListedColormap(['lightslategrey', 'coral', 'forestgreen']) #Define a palette
plt.style.use("classic")
plt.imshow(landc)
plt.title("Land Cover")
plt.colorbar()
plt.show()


plt.imshow(landc_suitability, cmap=cmap_suitability)
plt.title("Suitability classes: landcover")
plt.show()

![](landcover.png)
![](landc-suitability.png)

In [ ]:
## Physiographic analysis

The physiographic analysis looked at suitable conditions in terms of slope and aspect. South and south-east expositions are considered not suitable for the growth of avocado.

The following slope and aspect were again raterised to 60m for the analysis.

The slope file contains following classes:
1 as <2% slope
2 as 2%-5% slope
3 as 5%-15% slope
4 as >15% slope


The aspect file contains following classes:
1 as Plat
2 as N et NE
3 as E et SE
4 as S et SW
5 as W et NW

In [11]:
#Define the thresholds for slope 
# 3: 5%-15%, 4: >15%, 1:<2%, 2:2%-5%

slope_suitability = slope.copy()
slope_suitability[slope_suitability == 1] = 20 #Suitable
slope_suitability[(slope_suitability ==2) | (slope_suitability ==3)] = 10 #Moderate Suitable
slope_suitability[slope_suitability == 4] = 0 #Not Suitable

#Export results
saveRaster(ref_raster, "./Suitability/slope_suitability.tif", slope_suitability)

"""Display results"""
plt.style.use("classic")
plt.imshow(slope)
plt.title("Land topography: slope")
plt.colorbar()
plt.show()


plt.imshow(slope_suitability, cmap=cmap_suitability)
plt.title("Suitability classes: slope")
plt.show()

![](slope.png)
![](slope_suitability.png)

In [13]:
#Define the thresholds for aspect 
#Areas oriented to south and south-east are not suitable
#Which corresponds to the following degrees:
#1 as Plat
#2 as N et NE
#3 as E et SE
#4 as S et SW
#5 as W et NW

exposure_suitability = exposure.copy()
exposure_suitability[(exposure_suitability == 3)] = 0 #Not suitable
exposure_suitability[(exposure_suitability == 4)] = 10 #mediumly suitable
exposure_suitability[(exposure_suitability == 1) | (exposure_suitability == 2) | (exposure_suitability == 5)] = 20 #highly suitable

#Export results
saveRaster(ref_raster, "./Suitability/exposure_suitability.tif", exposure_suitability)

In [ ]:
"""Display results"""
plt.style.use("classic")
plt.imshow(exposure)
plt.title("Land topography: Exposure")
plt.colorbar()
plt.show()

plt.imshow(exposure_suitability, cmap=cmap_suitability)
plt.title("Suitability classes: exposure")
plt.show()

![](exposure.png)
![](exposure_suitability.png)

## Soil Parameter Analysis
The soil parameters are also obtained from the project.
Two soil properties were used 1: Texture and 2. Drainage

The optimal requirement for Avocado in terms of texture were coarse to balanced texture and
for drainage was medium to high.


In [15]:
#Define the thresholds for drainage 
#1: Barrage/barrage 
#2: Bon/well 
#3: Faible/low 
#4: Mauvais/bad 
#5: Sol squelettique/Skeletal soil 
#6: Zone Urbaine/urban area
drainage_suitability = drainage.copy()
drainage_suitability[(drainage_suitability == 4) | (drainage_suitability == 1) | (drainage_suitability == 6)] = 0 #Not suitable
drainage_suitability[(drainage_suitability == 5) | (drainage_suitability == 3)] = 10 #mediumly suitable
drainage_suitability[(drainage_suitability == 2)] = 20 #highly suitable

#Export results
saveRaster(ref_raster, "./Suitability/drainage_suitability.tif", drainage_suitability)

In [ ]:
"""Display results"""

cmap_suitability = colors.ListedColormap(['lightslategrey', 'coral', 'forestgreen']) #Define a palette
plt.imshow(drainage_suitability, cmap=cmap_suitability)
plt.title("Suitability classes: Drainage")
plt.show()

plt.style.use("classic")
plt.imshow(drainage)
plt.title("Soil Properties: Drainage")
plt.colorbar()
plt.show()

![](drainage.png)
![](drainage_suitability.png)

In [ ]:
#Define the thresholds for texture
#1: Argileuse/Clay 
#2: Argileuse/Equilibrée/Balanced/Clay
#3: Barrage/Barrage 
#4: Equilibrée/Balanced 
#5: Sableuse/Sand Blaster 
#6: Sol squelettique/Skeletal soil
#7: Zone Urbaine/Urban Area 

texture_suitability = texture.copy()
texture_suitability[(texture_suitability == 2) | (texture_suitability==4)] = 20 #Suitable
texture_suitability[(texture_suitability ==6)] = 10 #Moderate Suitable
texture_suitability[(texture_suitability == 5) | (texture_suitability==7) | (texture_suitability==1) | (texture_suitability==3)] = 0 #Not Suitable

#Export results
saveRaster(ref_raster, "./Suitability/texture_suitability.tif", texture_suitability)

"""Display results"""
plt.style.use("classic")
plt.imshow(texture)
plt.title("Soil properties: Texture")
plt.colorbar()
plt.show()

plt.imshow(texture_suitability, cmap=cmap_suitability)
plt.title("Suitability classes: texture")
plt.show()

![](texture.png)
![](texture_suitability.png)

## Combining layers
Finally, all the land cover, terrain and soil layers were overlaid and a suitability later is generated indicating higher value as more suitable layer.
Unsuitable layer is generated from the condition where atleast one layer/pixel is unsuitable for Avocado.

In [17]:
"""Combine layers"""
#Any pixel where at least one layer is unsuitable is set to unsuitable
test = exposure_suitability.copy() 
test[landc_suitability == 0] = 0
test[texture_suitability == 0] = 0
test[drainage_suitability == 0] = 0
test[slope_suitability == 0] = 0

combined = exposure_suitability + slope_suitability + landc_suitability + texture_suitability + drainage_suitability
result = np.where(test > 0, combined, 0 )

#Export results
saveRaster(ref_raster, "./Suitability/result.tif", result)

plt.imshow(result)
plt.title("Combined suitability")
plt.colorbar()
plt.show()

![](combined_suitability.png)

## Map showing average land suitability percentage through various assessments
![](compare_all.png)